In [1]:
import pandas as pd
import numpy as np

### Import the data in csv format

In [7]:
data=pd.read_csv('online_retail.csv')
data.head()

,InvoiceNo,StockCode,Description
0,562583,35637A,IVORY STRING CURTAIN WITH POLE
1,562583,35638A,PINK AND BLACK STRING CURTAIN
2,562583,84927F,PSYCHEDELIC TILE HOOK
3,562583,22425,ENAMEL COLANDER CREAM
4,562583,16008,SMALL FOLDING SCISSOR(POINTED EDGE)


### Checking the dataset
As it can be seen that, each row is sale of an item with similar Invoice No. So, we have to group all the items brought by the customer on the same invoice number. Lets keep StockCode for mining the rule then we can replace the StockCode with the Desciption of the item.

In [8]:
new=data.groupby('InvoiceNo')['StockCode'].apply(list)
transanctions=list(new)
#print(transanctions)

Total number of unique items in the dataset are:

In [9]:
len(data.StockCode.unique())

3353

### Movie Lense dataset
Lets import and see the movie lense dataset.

In [10]:
movie_data=pd.read_csv('movielens_movies.csv')
movie_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


So the assoication in the movie dataset would be done on the genres. So, we have to add generes as the tranasaction data for better analysis.

In [11]:
#extract on the basis of the | character.
movie_transanctions=movie_data['genres'].apply(lambda t: t.split('|'))
#convert this as the list
movie_transanctions=list(movie_transanctions)

Lets print and see the first transaction

In [12]:
print(movie_transanctions[0])

['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']


### Preparing the dataset for Association Rule Mining

### Online Retail Dataset

In [15]:
#The online retail dataset
#import the package for creation of one-hot encoder
from mlxtend.preprocessing import TransactionEncoder
#Get unique values from the dataset for encoding
encoderRetail=TransactionEncoder().fit(transanctions)
#Create one hot encoder
oneHotRetail=encoderRetail.transform(transanctions)
#Create the dataframe
oneHotRetail=pd.DataFrame(oneHotRetail, columns=encoderRetail.columns_)
print(oneHotRetail)

      10080  10120  10124A  10124G  10125  10133  10135  11001  15030  15034  \
0     False  False   False   False  False  False  False  False  False  False   
1     False  False   False   False  False  False  False  False  False  False   
2     False  False   False   False  False  False  False  False  False  False   
3     False  False   False   False  False  False  False  False  False  False   
4     False  False   False   False  False  False  False  False  False  False   
...     ...    ...     ...     ...    ...    ...    ...    ...    ...    ...   
9704  False  False   False   False  False  False  False  False  False  False   
9705  False  False   False   False  False  False  False  False  False  False   
9706  False  False   False   False  False  False  False  False  False  False   
9707  False  False   False   False  False  False  False  False  False  False   
9708  False  False   False   False  False  False  False  False  False  False   

      ...  DCGSSBOY  DCGSSGIRL    DOT  

Calculate the support value

In [17]:
oneHotRetail.mean()

10080           0.001545
10120           0.001545
10124A          0.000103
10124G          0.000206
10125           0.001133
                  ...   
POST            0.007519
S               0.000721
gift_0001_10    0.000412
gift_0001_20    0.000103
gift_0001_30    0.000412
Length: 3353, dtype: float64

### Movie Lense Dataset

In [16]:
encoderMovie=TransactionEncoder().fit(movie_transanctions)
oneHotMovie=encoderMovie.transform(movie_transanctions)
oneHotMovie=pd.DataFrame(oneHotMovie, columns=encoderMovie.columns_)
print(oneHotMovie)

       (no genres listed)  Action  Adventure  Animation  Children  Comedy  \
0                   False   False       True       True      True    True   
1                   False   False       True      False      True   False   
2                   False   False      False      False     False    True   
3                   False   False      False      False     False    True   
4                   False   False      False      False     False    True   
...                   ...     ...        ...        ...       ...     ...   
27273               False   False      False      False     False    True   
27274               False   False      False      False     False    True   
27275               False   False       True      False     False   False   
27276                True   False      False      False     False   False   
27277               False   False       True      False     False   False   

       Crime  Documentary  Drama  Fantasy  Film-Noir  Horror   IMAX  Musica

Computing the support value

In [18]:
oneHotMovie.mean()

(no genres listed)    0.009018
Action                0.129042
Adventure             0.085380
Animation             0.037649
Children              0.041755
Comedy                0.306987
Crime                 0.107743
Documentary           0.090586
Drama                 0.489185
Fantasy               0.051763
Film-Noir             0.012098
Horror                0.095718
IMAX                  0.007185
Musical               0.037979
Mystery               0.055503
Romance               0.151294
Sci-Fi                0.063898
Thriller              0.153164
War                   0.043772
Western               0.024782
dtype: float64